### Milestone 2 Trino

#### Federated Queries

In [9]:
URL="http://10.128.0.4:8080"

Notes: shopper-mysql / reservations-postgres / ticketing-mongo / currency-bigq

##### Q1: Who are the shoppers who have reservations in a different city from their own city and who paid for their reservations in US dollars? For those shoppers, return their cust_id, email, along with their reservation count. Order the results by cust_id. Limit the results to 5 records.


The query we use searches for shoppers who had reservations in a different city from their own city and paid for their reservations in Euros instead of USD since there were no records that matched the requirements of USD but there were two who matched using Euros, thus we decided to use EUR instead to show that our code works.

In [108]:
!trino $URL --execute "select s.cust_id, s.email, count(*) as reservation_count from mysql.final_project.shopper s\
join postgres.final_project.reservations r on (s.cust_id = r.cust_id and s.city != r.city)\
join bigquery.final_project.currency c on r.cnt_code = c.cntry_code and c.curr_code = 'EUR'\
group by s.cust_id, s.email\
order by s.cust_id\
limit 5"

"36","Jessica.Elliott@nguyen-meyer.biz","1"
"43","Andrew.Garcia@joseph.com","1"


##### Q2: Who are the shoppers who have bought non-stop tickets on American Airlines with a departure date between 01/01/2020 and 12/31/2024? Return the shopper’s cust_id, email, dep_airport and dep_date. Order the results by dep_date. Limit the results to 5 records.

The query returns the cust_id, email, departure airport, and departure date for shoppers who have bought non-stop tickets on American Airlines with in the date range given. The results are limited to 5, but there is only one result due to the absence of records that match the requirements in the randomized data. 

In [49]:
!trino $URL --execute "SELECT s.cust_id, s.email, t.dep_airport, t.dep_date FROM mysql.final_project.shopper s\
JOIN mongo.final_project.ticketing t ON s.cust_id = t.cust_id\
WHERE t.airline = 'American Airlines'\
AND CAST(t.dep_date AS date) BETWEEN DATE '2020-01-01' AND DATE '2024-12-31'\
AND t.stops = 0\
ORDER BY t.dep_date\
LIMIT 5"

"40","Stephen.Frazier@conley-krueger.com","Bangkok International airport","2020-03-24 05:50:08.000"


##### Q3: Which shoppers have reservations and tickets with matching dates such that the arr_date and dep_date are equal between reservations and ticketing? For such shoppers, sum up their pmt_amt and tik_amt and return this sum as trip_amount. Also, return the shoppers’ cust_id, email, pmt_amt, tik_amt. Order the results by trip_amount in descending order. Limit the results to 5 records.

Top cell is the full query including the restriction on reservation and tickets dates while the bottom cell is the same query without the date restrictions, the top cell does not return any output since the date restrictions are too strict in randomized data, so we intended to show that our code does work in the bottom cell.

In [56]:
!trino $URL --execute "select s.cust_id, s.email, COALESCE(SUM(r.pmt_amt) + SUM(t.tik_amt), 0) AS trip_amount from mysql.final_project.shopper s \
join postgres.final_project.reservations r on s.cust_id = r.cust_id \
join mongo.final_project.ticketing t on s.cust_id = t.cust_id \
where r.arr_date = t.arr_date and r.dep_date = t.dep_date\
group by s.cust_id, s.email\
order by s.cust_id DESC\
limit 5"

In [57]:
!trino $URL --execute "select s.cust_id, s.email, COALESCE(SUM(r.pmt_amt) + SUM(t.tik_amt), 0) AS trip_amount from mysql.final_project.shopper s \
join postgres.final_project.reservations r on s.cust_id = r.cust_id \
join mongo.final_project.ticketing t on s.cust_id = t.cust_id \
group by s.cust_id, s.email\
order by s.cust_id DESC\
limit 5"

"50","Melissa.Reynolds@walker.com","5117.50"
"47","Joseph.Ramirez@daniel.com","8220.50"
"46","Anna.Russell@thomas.net","1961.18"
"45","Brittany.Jarvis@clarke.com","4354.51"
"44","David.Mercado@mays.com","5731.32"


#### Trino Evaluation

Now that you have seen how to implement cross-database joins, it’s time to conduct a more
thorough evaluation of Trino. Think of other tests you can run to gain a more complete picture of
Trino’s functionality. Come up with some tests and run them from your notebook. You want to
test a variety of scenarios that include DDL, DML, and SELECT statements. Then, write a short
paragraph to summarize your findings.

Your summary needs to be specific and supported by your test results. For example: “CREATE
TABLE statement X against MySQL resulted in Y” or “BEGIN TRANSACTION and COMMIT”
against Postgres resulted in Z”. Write your summary as a Markdown comment on the last cell of
the notebook.

##### MySQL

Creates new_shoppers in mysql.final_project as a copy of the shopper table

In [61]:
!trino $URL --execute "CREATE TABLE mysql.final_project.new_shoppers AS SELECT * FROM mysql.final_project.shopper"
!trino $URL --execute "select * from mysql.final_project.new_shoppers limit 10"

CREATE TABLE: 50 rows
"1","Tammy","Alexander","Gordon and Sons","871 Zachary Cape","8147","Josephborough","Mcdowelltown","SE","10040","0527620197218","2051824706402","Tammy.Alexander@davis.com"
"2","Joseph","Evans","Calderon Group","435 Jacqueline Valleys Apt. 942","571","South Jasonport","East Williamland","ME","72631","4719416316378","5386293405827","Joseph.Evans@graham-wilson.net"
"3","Rebecca","Tapia","Bonilla Ltd","8465 Mary Course Suite 466","481","Hillport","Williamsmouth","HT","76394","2640176467267","4959532897707","Rebecca.Tapia@williams.com"
"4","Donna","Garcia","Adams-Malone","45206 Elizabeth Cape","25856","Port Katherineport","Helenshire","GN","50857","8109593277289","2832886528989","Donna.Garcia@steele.com"
"5","Carol","Griffin","Fischer, Allen and Joyce","595 Thompson Route","33998","New Tara","Deniseton","US","39944","5954665340594","8903899582545","Carol.Griffin@simmons-peterson.info"
"6","Mary","Nelson","Wallace, Cross and Fitzgerald","869 Daniel Mountains","1882","Jo

Drops the state column from the new_shoppers table created in the previous query. It can be seen that state is absent when comparing the output to the previous output. 

In [63]:
!trino $URL --execute "ALTER TABLE mysql.final_project.new_shoppers DROP COLUMN state"
!trino $URL --execute "select * from mysql.final_project.new_shoppers limit 10"

DROP COLUMN
"1","Tammy","Alexander","Gordon and Sons","871 Zachary Cape","8147","Josephborough","Mcdowelltown","10040","0527620197218","2051824706402","Tammy.Alexander@davis.com"
"2","Joseph","Evans","Calderon Group","435 Jacqueline Valleys Apt. 942","571","South Jasonport","East Williamland","72631","4719416316378","5386293405827","Joseph.Evans@graham-wilson.net"
"3","Rebecca","Tapia","Bonilla Ltd","8465 Mary Course Suite 466","481","Hillport","Williamsmouth","76394","2640176467267","4959532897707","Rebecca.Tapia@williams.com"
"4","Donna","Garcia","Adams-Malone","45206 Elizabeth Cape","25856","Port Katherineport","Helenshire","50857","8109593277289","2832886528989","Donna.Garcia@steele.com"
"5","Carol","Griffin","Fischer, Allen and Joyce","595 Thompson Route","33998","New Tara","Deniseton","39944","5954665340594","8903899582545","Carol.Griffin@simmons-peterson.info"
"6","Mary","Nelson","Wallace, Cross and Fitzgerald","869 Daniel Mountains","1882","Johnland","Philipport","39057","68397

Inserts a new record into the new_shoppers table, the new record doesn't have a state field since the column was dropped in the previous query. The select statement shows the inserted record and a record from the original table for comparison

In [117]:
!trino $URL --execute "INSERT INTO mysql.final_project.new_shoppers VALUES (51, 'Tony', 'Wang', 'UTexas', '110 Inner Campus Drive', '143', 'Austin', 'Travis', '78712', '0017136514777', '0037126554777', 'randomemail@gmail.com')"
!trino $URL --execute "select * from mysql.final_project.new_shoppers where cust_id = 51"
!trino $URL --execute "select * from mysql.final_project.shopper where cust_id = 1"

INSERT: 1 row
"51","Tony","Wang","UTexas","110 Inner Campus Drive","143","Austin","Travis","78712","0017136514777","0037126554777","randomemail@gmail.com"
"1","Tammy","Alexander","Gordon and Sons","871 Zachary Cape","8147","Josephborough","Mcdowelltown","SE","10040","0527620197218","2051824706402","Tammy.Alexander@davis.com"


In [118]:
!trino $URL --execute "DELETE FROM mysql.final_project.new_shoppers where cust_id = 51"

DELETE: 1 row


Updates the city field of all records with cust_id greater than 40 to Houston   

In [66]:
!trino $URL --execute "UPDATE mysql.final_project.new_shoppers SET city = 'Houston' WHERE cust_id > 40"
!trino $URL --execute "select * from mysql.final_project.new_shoppers where cust_id > 40"

UPDATE: 11 rows
"41","Erik","Cole","Kennedy, Burton and Hickman","44055 Hannah Vista","25585","Houston","Kevinville","57292","9409051117106","5726071115794","Erik.Cole@barnes.com"
"42","Eric","Moore","Atkinson Inc","72549 Kemp Islands","855","Houston","West Michelle","67208","9158420610977","8477647830512","Eric.Moore@martinez.com"
"43","Andrew","Garcia","Hughes and Sons","513 Calderon Circle","1958","Houston","Chanburgh","11652","5357802342809","8947721315514","Andrew.Garcia@joseph.com"
"44","David","Mercado","Manning-Grant","04997 Sue Parkways Apt. 735","2831","Houston","New Erik","23171","9641613952596","3080626176897","David.Mercado@mays.com"
"45","Brittany","Jarvis","Lyons-Martinez","7102 Mackenzie Park","62931","Houston","Adamsburgh","25027","2222844338640","2119135279620","Brittany.Jarvis@clarke.com"
"46","Anna","Russell","Leblanc, Bailey and White","575 Benitez Mission Apt. 855","88822","Houston","Saraborough","67277","6090262019122","5030912872340","Anna.Russell@thomas.net"
"4

##### Postgres

Creates new_reservations in postgres.final_project as a copy of the reservations table

In [104]:
!trino $URL --execute "CREATE TABLE postgres.final_project.new_reservations AS SELECT * FROM postgres.final_project.reservations"
!trino $URL --execute "select * from postgres.final_project.new_reservations limit 10"

CREATE TABLE: 100 rows
"1","50","let","but","90115 Andre Turnpike","Apt. 597","Hernandezburgh","VT","15497","-86.075802","90.090998","RU","2023-06-16","2020-12-24","481.47"
"2","23","exactly","enjoy","8081 Matthew Station","Suite 620","New Tylerland","OH","28693","-51.279333","-165.435410","LY","2023-01-10","2023-10-04","470.64"
"3","43","sure","amount","2427 Anna Track","Suite 759","Alvarezfort","VI","78477","-44.373662","56.006519","ES","2023-01-25","2023-03-14","817.64"
"4","42","dark","hair","294 Melanie Stravenue","Suite 711","Lewisport","AS","05481","-31.107518","92.040456","BT","2023-02-10","2023-09-08","631.63"
"5","46","various","very","353 Thompson Prairie Apt. 405","Apt. 787","Dominicton","RI","22138","-51.096046","113.166226","BJ","2022-10-20","2020-09-02","123.06"
"6","26","since","garden","648 Henderson Spur","Apt. 814","Robertshire","KY","69418","-59.468444","21.513665","IS","2020-11-02","2022-10-24","640.17"
"7","26","fire","laugh","67249 Rios Key Apt. 289","Suite 707",

Runs three DML statements in one block of code, originally intended to be a transaction statement but we found out that postgres in trino uses autocommits. The code inserts a new record into the new_reservations table, then updates the cnt_code of red_id < 3 to 'CN', and finally deletes the record with res_id = 4. The select query shows the updated country codes, the inserted record, and the absense of the dropped record.

In [105]:
!trino $URL --execute "INSERT INTO postgres.final_project.new_reservations \
VALUES (101, 51, 'Fast', 'Food', '110 Inner Campus Drive', '143', 'Austin', 'TX', '78712', -86.075202, 90.090398, 'US', DATE '2023-06-12', DATE ' 2023-06-15', 100.32); \
UPDATE postgres.final_project.new_reservations \
SET cnt_code = 'CN' \
WHERE res_id < 3; \
DELETE FROM postgres.final_project.new_reservations \
WHERE res_id = 4;"
!trino $URL --execute "select * from postgres.final_project.new_reservations where res_id = 101 or res_id < 6"

INSERT: 1 row
UPDATE: 2 rows
DELETE: 1 row
"3","43","sure","amount","2427 Anna Track","Suite 759","Alvarezfort","VI","78477","-44.373662","56.006519","ES","2023-01-25","2023-03-14","817.64"
"5","46","various","very","353 Thompson Prairie Apt. 405","Apt. 787","Dominicton","RI","22138","-51.096046","113.166226","BJ","2022-10-20","2020-09-02","123.06"
"101","51","Fast","Food","110 Inner Campus Drive","143","Austin","TX","78712","-86.075202","90.090398","US","2023-06-12","2023-06-15","100.32"
"1","50","let","but","90115 Andre Turnpike","Apt. 597","Hernandezburgh","VT","15497","-86.075802","90.090998","CN","2023-06-16","2020-12-24","481.47"
"2","23","exactly","enjoy","8081 Matthew Station","Suite 620","New Tylerland","OH","28693","-51.279333","-165.435410","CN","2023-01-10","2023-10-04","470.64"


Drops the new_reservations table 

In [ ]:
!trino $URL --execute "DROP TABLE postgres.final_project.new_reservations"
!trino $URL --execute "select * from postgres.final_project.new_reservations"

DROP TABLE
Query 20231206_183626_00062_stmdw failed: line 1:15: Table 'postgres.final_project.new_reservations' does not exist
select * from postgres.final_project.new_reservations



##### MongoDB

Creates new_ticketing in mongo.final_project as a copy of the ticketing table

In [158]:
!trino $URL --execute "CREATE TABLE mongo.final_project.new_ticketing AS SELECT * FROM mongo.final_project.ticketing"
!trino $URL --execute "select * from mongo.final_project.new_ticketing limit 10"

CREATE TABLE: 100 rows
"157cf9c6-b16e-4d5c-abeb-959208f0ebd4","38","Hainan Airlines","200","Harbin Yangjiagang airport","Marechal Cunha Machado International airport","2023-10-17 00:24:50.000","0001-01-01 00:24:50.000","2020-01-11 07:14:36.000","0001-01-01 07:14:36.000","0","747","MGA"
"2607ad5f-76ab-4475-9da6-18fd7bf78a4d","40","Air China","193","Mingaladon airport","Dubai International airport","2020-12-05 22:29:42.000","0001-01-01 22:29:42.000","2023-02-05 20:11:47.000","0001-01-01 20:11:47.000","0","857","GBP"
"e98d7c35-8a84-415c-aad1-426810872756","16","Southern Airways Express","577","Orio Al Serio airport","Shimojishima airport","2022-10-20 15:38:46.000","0001-01-01 15:38:46.000","2021-12-22 10:34:31.000","0001-01-01 10:34:31.000","1","522","BND"
"7b25331f-4d6b-4d8f-a506-bfc51025dbe5","36","LATAM Airlines","970","Okecie International airport","Cap C Martinez de Pinillos airport","2022-05-12 14:41:55.000","0001-01-01 14:41:55.000","2022-04-10 23:15:15.000","0001-01-01 23:15:15.00

In [157]:
!trino $URL --execute "DROP TABLE mongo.final_project.new_ticketing"

DROP TABLE


Drops the dep_time and arr_time column from the new_ticketing table created in the previous query. It can be seen that dep_time is absent when comparing the output to the previous output.

In [159]:
!trino $URL --execute "ALTER TABLE mongo.final_project.new_ticketing DROP COLUMN dep_time"
!trino $URL --execute "ALTER TABLE mongo.final_project.new_ticketing DROP COLUMN arr_time"
!trino $URL --execute "select * from mongo.final_project.new_ticketing limit 10"

DROP COLUMN
DROP COLUMN
"157cf9c6-b16e-4d5c-abeb-959208f0ebd4","38","Hainan Airlines","200","Harbin Yangjiagang airport","Marechal Cunha Machado International airport","2023-10-17 00:24:50.000","2020-01-11 07:14:36.000","0","747","MGA"
"2607ad5f-76ab-4475-9da6-18fd7bf78a4d","40","Air China","193","Mingaladon airport","Dubai International airport","2020-12-05 22:29:42.000","2023-02-05 20:11:47.000","0","857","GBP"
"e98d7c35-8a84-415c-aad1-426810872756","16","Southern Airways Express","577","Orio Al Serio airport","Shimojishima airport","2022-10-20 15:38:46.000","2021-12-22 10:34:31.000","1","522","BND"
"7b25331f-4d6b-4d8f-a506-bfc51025dbe5","36","LATAM Airlines","970","Okecie International airport","Cap C Martinez de Pinillos airport","2022-05-12 14:41:55.000","2022-04-10 23:15:15.000","1","814","MGA"
"121213ca-8212-47c6-b104-8aa604f0d0f2","43","Wings Air","105","Memphis International airport","Valencia airport","2021-01-17 21:15:53.000","2020-10-13 07:32:02.000","3","832","LSL"
"8ead31

Inserts a new record into the new_ticketing table, the new record doesn't have a dep_time or a arr_time field since the columns were dropped in the previous query. The select statement shows the inserted record and a record from the original table for comparison

In [162]:
!trino $URL --execute "INSERT INTO mongo.final_project.new_ticketing VALUES (cast('0000000000000000000000000000000' as varchar), cast(51 as bigint), cast('Spirit' as varchar), cast(777 as bigint), 'Dallas Fort Worth Airport', 'Austin Airport', TIMESTAMP '2020-06-10 15:55:23.383', TIMESTAMP '2020-06-10 15:55:23.383', 5, 700, 'USD')"
!trino $URL --execute "select * from mongo.final_project.new_ticketing where tck_id = '0000000000000000000000000000000'"
!trino $URL --execute "select * from mongo.final_project.ticketing where tck_id = '157cf9c6-b16e-4d5c-abeb-959208f0ebd4'"

INSERT: 1 row
"0000000000000000000000000000000","51","Spirit","777","Dallas Fort Worth Airport","Austin Airport","2020-06-10 15:55:23.383","2020-06-10 15:55:23.383","5","700","USD"
"157cf9c6-b16e-4d5c-abeb-959208f0ebd4","38","Hainan Airlines","200","Harbin Yangjiagang airport","Marechal Cunha Machado International airport","2023-10-17 00:24:50.000","0001-01-01 00:24:50.000","2020-01-11 07:14:36.000","0001-01-01 07:14:36.000","0","747","MGA"


note: uniqueness, timestamp, type conversion

Updates the tik_amt to 500 for tickets with flights that have one stop. This query failed.

In [168]:
!trino $URL --execute "UPDATE mongo.final_project.new_ticketing SET tik_amt = 500 WHERE stops = 1"
!trino $URL --execute "select * from mongo.final_project.new_ticketing where stops = 1 limit 5"

Query 20231207_010409_00002_stmdw failed: This connector does not support modifying table rows

"e98d7c35-8a84-415c-aad1-426810872756","16","Southern Airways Express","577","Orio Al Serio airport","Shimojishima airport","2022-10-20 15:38:46.000","2021-12-22 10:34:31.000","1","522","BND"
"7b25331f-4d6b-4d8f-a506-bfc51025dbe5","36","LATAM Airlines","970","Okecie International airport","Cap C Martinez de Pinillos airport","2022-05-12 14:41:55.000","2022-04-10 23:15:15.000","1","814","MGA"
"e1ae6a00-984c-4df8-913d-74f3dec4ac46","31","VietJet Air","274","Arlanda airport","Cariri Regional airport","2021-05-05 16:49:25.000","2021-11-13 10:12:22.000","1","306","PHP"
"1bb039fa-5868-499b-b100-7342a41b657a","17","Ural Airlines","819","Los Angeles International airport","Governador Andre Franco Montoro International Ai","2020-05-27 12:54:29.000","2021-06-24 15:13:04.000","1","412","EGP"
"3505c031-530c-499e-bc9a-000564357c97","28","Indonesia AirAsia","942","Otopeni airport","Franz-Josef-Strauss air

##### BigQuery

Creates new_currency in bigquery.final_project as a copy of the currency table

In [169]:
!trino $URL --execute "CREATE TABLE bigquery.final_project.new_currency AS SELECT * FROM bigquery.final_project.currency"
!trino $URL --execute "select * from bigquery.final_project.new_currency limit 10"

CREATE TABLE: 30 rows
"PHP","Renminbi","AF","Bermuda"
"ANG","Australian dollar","AM","Guatemala"
"JMD","Trinidad and Tobago dollar","BI","Oman"
"COP","Swazi lilangeni","CA","Switzerland"
"ARS","Moroccan dirham","CG","Zimbabwe"
"AFN","Lesotho loti","CV","Russian Federation"
"SAR","Mauritian rupee","DM","Austria"
"DOP","Belarusian ruble","EG","Rwanda"
"EUR","Philippine peso","ES","Slovakia (Slovak Republic)"
"BRL","Trinidad and Tobago dollar","GM","Cambodia"


This code block runs 3 DML statements. The code inserts a new record into the new_currency table, then updates the cntry_code of the new record to 'CN', and lastly deletes the new record based on curr_code. 2 out of the 3 statements failed.

In [179]:
!trino $URL --execute "INSERT INTO bigquery.final_project.new_currency \
VALUES ('ZZZ', 'Zebra Zollars', 'ZA', 'Zootopia')"
!trino $URL --execute "UPDATE bigquery.final_project.new_currency \
SET cntry_code = 'CN' WHERE curr_code = 'ZZZ';"
!trino $URL --execute "select * from bigquery.final_project.new_currency where curr_code = 'ZZZ'"
!trino $URL --execute "DELETE FROM bigquery.final_project.new_currency WHERE curr_code = 'ZZZ';"

INSERT: 1 row
Query 20231207_021658_00029_stmdw failed: This connector does not support modifying table rows

"ZZZ","Zebra Zollars","ZA","Zootopia"
"ZZZ","Zebra Zollars","ZA","Zootopia"
"ZZZ","Zebra Zollars","ZA","Zootopia"
"ZZZ","Zebra Zollars","ZA","Zootopia"
"ZZZ","Zebra Zollars","ZA","Zootopia"
"ZZZ","Zebra Zollars","ZA","Zootopia"
Query 20231207_021702_00031_stmdw failed: This connector does not support modifying table rows



Drops the new_currency table

In [183]:
!trino $URL --execute "DROP TABLE bigquery.final_project.new_currency"
!trino $URL --execute "select * from bigquery.final_project.new_currency"

DROP TABLE
Query 20231207_023823_00036_stmdw failed: line 1:15: Table 'bigquery.final_project.new_currency' does not exist
select * from bigquery.final_project.new_currency



#### Evaluation Summary

Using mysql, the first DDL statement we implemented was "CREATE TABLE mysql.final_project.new_shoppers AS SELECT * FROM mysql.final_project.shopper" to create a new_shoppers table that was a copy of the original shopper table from milestone 1. The result was as expected and trino created the table just as if it were run using mysql natively. The second DDL statement we implemented in mysql was "ALTER TABLE mysql.final_project.new_shoppers DROP COLUMN state" to drop the state column in the new table. The result was the state column being successfully dropped as expected. The two DML statements run were "INSERT INTO ... VALUES" and "UPDATE ... SET ... WHERE", both resulted in the statements being run successfully for its intended purpose like it was run natively using mysql. From the mysql statements, it seems that trino can execute basic DDL, DML, and select statements for mysql perfectly. 

Using postgres, we wrote three DML statements in a single transaction "BEGIN TRANSACTION ... INSERT INTO ... UPDATE ... DELETE ... COMMIT", which resulted in a mismatched input error where "BEGIN" is not a recognized keyword in trino. Then, when changing to the supported keyword "START TRANSACTION ... COMMIT". We then received the error that postgres only supports autocommit. We found out that each statement is treated as a single transaction within trino for postgres so there is no need to use BEGIN, START, or COMMIT. For the first DDL statement in postgres, we ran a "CREATE TABLE ... AS SELECT ..." in the exact same way as the mysql statement to create a new_reservations table as a copy of the reservations table. The second DDL statement we ran was "DROP TABLE ..." to drop the newly created new_reservations table. Both DDL statements ran without error just like it would if it were run in native postgres. From the postgres statements, we learned that trino though trino can run sql query statements perfectly, it doesn't have much support for running statements in transactions. 

Using mongodb, the we implemented "CREATE TABLE mongo.final_project.new_ticketing AS SELECT * FROM mongo.final_project.ticketing" to create a new_ticketing table, which was a copy of the original ticketing table. As expected, trino created the table very easily. Then for the second DDL statements, we used "ALTER TABLE mongo.final_project.new_ticketing DROP COLUMN" to drop dep_time and arr_time in the new table. These columns successfully dropped. The two DML statements were "INSERT INTO ... VALUES" and "UPDATE ... SET ... WHERE". The former statement ran successfully after accounting for the fact that trino will not convert between character and numeric types, and accounting for the datetime to timestamp conversion. The latter statement, suprisingly failed, and gave an error that said "This connector does not support modifying table rows." We concluded it is not possible to update records in tables in mongodb through trino with the given setup we have.

Using bigquery, we implemented the first DDL statement which was "CREATE TABLE bigquery.final_project.new_currency AS SELECT * FROM bigquery.final_project.currency". This created a new table containing the same contents as the original currency table. This ran successfully as expected. Next we ran our DML statements, which were "INSERT INTO ...", "UPDATE ...", and "DELETE ...", and found that while insert ran, both update and delete failed. The bigquery connector also seems to not support modifying tables like mongodb. The last DDL statement was "DROP TABLE ..." to drop the new_currency table. This reinforces the fact that while trino can run sql query statements smoothly, there is much to be improved on for transactions.

In the end, trino was designed to query large data sets distributed over one or more heterogeneous data sources. With this in mind, trino performs very well. So if a user would like to have an easier experience in query federation, then trino is a fine query engine to use, but keep in mind a consequence is less support on transactional statements.
